In [1]:
import numpy as _np
import spinmob as _s
import matplotlib.pyplot as _plt
import os as _os
from scipy.optimize import curve_fit

In [2]:
data_file_names = []
betas = []
Ns = []
beta_start = "beta_"
beta_end = "_par"
N_start = "N_"
N_end = "_beta"
for name in _os.listdir(_os.getcwd()+"\\problem3data"):
    if name.find("beta")>=0:
        data_file_names.append(_os.getcwd()+"\\problem3data\\"+name)
        betas.append(float(name[name.find(beta_start)+len(beta_start):name.find(beta_end)]))
        Ns.append(int(name[name.find(N_start)+len(N_start):name.find(N_end)]))
print(data_file_names)
print(betas)
print(Ns)


['c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.88_parallel(1).csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.88_parallel.csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.89_parallel(1).csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.89_parallel.csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.91_parallel(1).csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.91_parallel.csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.92_parallel(1).csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.92_parallel.csv', 'c:\\Users\\MWStr\\code\\phys765_advanced_quantum\\project2\\problem3data\\N_16_beta_0.93_parallel(1).csv', 'c:\\Users\\MWStr\\code\\phys765_advanc

In [3]:
data_boxes = _s.data.load_multiple(paths = data_file_names)

In [4]:
len(data_boxes)

48

In [5]:
def exp_decay(x, gamma, amp):
    return amp*_np.exp(-x/gamma)

def correlation(data, data_average):
    data_correlations = []
    correlation_distance = _np.array(range(len(data)-1))+1
    for distance in correlation_distance:
        correlations = data[:-distance]*data[distance::]-data_average**2
        data_correlations.append(_np.average(correlations))
    return data_correlations, correlation_distance

def fit_exp_decay(data, distacnes, p0):
    popt, cov = curve_fit(exp_decay, distacnes[:2000], data[:2000], p0=p0)
    return [popt, cov]

def plot_exp_decay_fit(correlation, distances, popt, ep_str, n_or_e):
    _plt.plot(distances, correlation, label="data")
    _plt.axhline(0, linestyle="--", color="grey", alpha=0.5)
    _plt.plot(distances[:2000], exp_decay(distances[:2000], *popt), label="fit")
    _plt.title(n_or_e+" correlation for ε="+ep_str, size=35)
    _plt.ylabel(r'$\tilde{e_n}\tilde{e}_{n+d} - \langle \tilde{e}\rangle^2$', size=35)
    _plt.xlabel("d", size=35)
    _plt.tick_params(axis="both", labelsize=30)
    _plt.legend(fontsize=30)
    _plt.show()

def choose_unc(es, es_popt, ns, ns_popt):
    es_uncs = []
    ns_uncs = []
    num_trials = len(es)
    for position, block in enumerate(block_sizes):
        blocked_es = _np.reshape(es, (int(num_trials/block), block))
        block_avg_es = _np.average(blocked_es, axis=1)
        unc = _np.std(block_avg_es)/_np.sqrt(len(block_avg_es))
        es_uncs.append(unc)

        blocked_ns = _np.reshape(ns, (int(num_trials/block), block))
        block_avg_ns = _np.average(blocked_ns, axis=1)
        unc = _np.std(block_avg_ns)/_np.sqrt(len(block_avg_ns))
        ns_uncs.append(unc)

        print("E Unc: ", es_uncs[-1], "E correlation/unc: ", _np.sqrt(exp_decay(block, *es_popt))/unc)
        print("n Unc: ", ns_uncs[-1], "n correlation/unc: ", _np.sqrt(exp_decay(block, *ns_popt))/unc)
        print(" num blocks: ", len(block_avg_es), " blocksize: ", block, "choice number", position)
        print("\r\r\r")
    which_block = int(input("Which block_size?"))
    return [block_sizes[which_block], es_uncs[which_block], ns_uncs[which_block]]


In [6]:
ns0 = data_boxes[2][0][1000:]
ns0_average = _np.average(ns0)
ns0_correlation, ns0_distances = correlation(ns0, ns0_average)
p0 = [200, 30]
ns0_popt, ns0_cov = fit_exp_decay(ns0_correlation, ns0_distances, p0)
plot_exp_decay_fit(ns0_correlation, ns0_distances, ns0_popt, "", r"$\chi$")
print(Ns[0])
ns0_popt


16


array([ 14.45406099, 413.4934241 ])

In [7]:
block_size = 100
exp_decay(block_size, *ns0_popt)/ns0_average*100

0.12314666333433891

In [8]:
def calcualte_windings_avg(data_box):
    return _np.average(data_box[2])
def calcualte_windings_unc(data_box):
    return _np.std(data_box[2])/_np.sqrt(len(data_box[2])/block_size)

windings_avg = [calcualte_windings_avg(data_box) for data_box in data_boxes]
windings_unc = [calcualte_windings_unc(data_box) for data_box in data_boxes]

In [9]:
betas_final = []
Ns_final = []
chis_16 = []
chis_unc_16 = []
chis_20 = []
chis_unc_20 = []
chis_24 = []
chis_unc_24 = []
N=20

def avg(avgs_array, uncs_array, indexes):
    avgs = _np.array(avgs_array)[indexes]
    uncs = _np.array(uncs_array)[indexes]
    avgs = avgs[_np.nonzero(uncs)]
    uncs = uncs[_np.nonzero(uncs)]
    weights = uncs**-2
    weighted_average = (_np.sum(weights*avgs)/_np.sum(weights))
    final_unc = (1/_np.sqrt(_np.sum(weights)))
    return weighted_average, final_unc


for beta in set(betas):
    betas_final.append(beta)
    Ns_final.append(N)
    indexes_16 = _np.nonzero(_np.logical_and(_np.array(betas)==beta, _np.array(Ns)==16))
    indexes_20 = _np.nonzero(_np.logical_and(_np.array(betas)==beta, _np.array(Ns)==20))
    indexes_24 = _np.nonzero(_np.logical_and(_np.array(betas)==beta, _np.array(Ns)==24))
    print(indexes_16, indexes_20, indexes_24)
    avg_16, unc_16 = avg(windings_avg, windings_unc, indexes_16)
    avg_20, unc_20 = avg(windings_avg, windings_unc, indexes_20)
    avg_24, unc_24 = avg(windings_avg, windings_unc, indexes_24)
    chis_16.append(avg_16)
    chis_unc_16.append(unc_16)
    chis_20.append(avg_20)
    chis_unc_20.append(unc_20)
    chis_24.append(avg_24)
    chis_unc_24.append(unc_24)

sort_indexes = _np.argsort(betas_final)
betas_final = _np.array(betas_final)[sort_indexes]
chis_16 = _np.array(chis_16)[sort_indexes]
chis_unc_16 = _np.array(chis_unc_16)[sort_indexes]
chis_20 = _np.array(chis_20)[sort_indexes]
chis_unc_20 = _np.array(chis_unc_20)[sort_indexes]
chis_24 = _np.array(chis_24)[sort_indexes]
chis_unc_124 = _np.array(chis_unc_24)[sort_indexes]
print(betas_final)
print(chis_16)
print(chis_unc_16)
print(chis_20)
print(chis_unc_20)
print(chis_24)
print(chis_unc_24)

(array([0, 1], dtype=int64),) (array([16, 17], dtype=int64),) (array([32, 33], dtype=int64),)
(array([2, 3], dtype=int64),) (array([18, 19], dtype=int64),) (array([34, 35], dtype=int64),)
(array([6, 7], dtype=int64),) (array([22, 23], dtype=int64),) (array([38, 39], dtype=int64),)
(array([4, 5], dtype=int64),) (array([20, 21], dtype=int64),) (array([36, 37], dtype=int64),)
(array([8, 9], dtype=int64),) (array([24, 25], dtype=int64),) (array([40, 41], dtype=int64),)
(array([10, 11], dtype=int64),) (array([26, 27], dtype=int64),) (array([42, 43], dtype=int64),)
(array([12, 13], dtype=int64),) (array([28, 29], dtype=int64),) (array([44, 45], dtype=int64),)
(array([14, 15], dtype=int64),) (array([30, 31], dtype=int64),) (array([46, 47], dtype=int64),)
[0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95]
[0.22079667 0.2706387  0.34868733 0.43341478 0.43175462 0.5229698
 0.61620204 0.79800292]
[0.01430051 0.01621802 0.01848305 0.02178716 0.0221155  0.02457274
 0.0269932  0.03327961]
[0.1567752  0.22106

In [10]:
_plt.errorbar(betas_final, chis_16, yerr=chis_unc_16, marker=".", color="blue", zorder=1, label="N=16")
_plt.errorbar(betas_final, chis_20, yerr=chis_unc_20, marker=".", color="green", zorder=1, label="N=20")
_plt.errorbar(betas_final, chis_24, yerr=chis_unc_24, marker=".", color="purple", zorder=1, label="N=24")
_plt.title(r"$\chi_\omega \: vs \: \beta \: with \: \epsilon=0.01$", size=18)
_plt.xlabel(r"$\beta$", size=18)
_plt.ylabel(r"$\chi_\omega$", size=18)
_plt.legend()

In [18]:
chis_unc/chis*100

array([4.91648702, 8.44561809, 3.35555935, 2.37983133, 1.41264976,
       1.03475354, 0.68827289, 0.56780386, 0.47833158, 0.46641142,
       0.4428813 , 0.44908512, 0.44854535, 0.43894373, 0.44320523])

In [11]:
_np.average([0.925, 0.9275,0.9313])

0.9279333333333334

In [12]:
_np.std([0.925, 0.9275,0.9313])

0.0025901522906749794

In [18]:
for jj in range(len(chis_16)):
    print(betas_final[jj], chis_16[jj], chis_unc_16[jj], "||", chis_20[jj], chis_unc_20[jj], "||", chis_24[jj], chis_unc_24[jj])

0.88 0.2207966745030477 0.01430050922056269 || 0.1567751961416103 0.011600518640511051 || 0.11105254776110972 0.009532428555137078
0.89 0.27063870008170077 0.01621802043636162 || 0.22106306614550278 0.014326938812429552 || 0.16613307119930446 0.012008734287425327
0.9 0.3486873313627744 0.01848304580563656 || 0.30463098333600575 0.017302725616332146 || 0.25506836795948756 0.0197467905943788
0.91 0.43341478018609125 0.02178715584927068 || 0.4078379279757144 0.0213150516447774 || 0.370411218088683 0.019950164398290105
0.92 0.43175461870676224 0.022115498952385182 || 0.4013647938441588 0.020588957304421986 || 0.3810010942576526 0.02537832184232094
0.93 0.5229697976393621 0.024572744448488976 || 0.5188569623787315 0.024322383058461003 || 0.539008187017439 0.030665632407949162
0.94 0.6162020415119904 0.026993200278815258 || 0.642601968975964 0.028860961857291343 || 0.7145508416792828 0.042229461973366064
0.95 0.7980029243125601 0.03327961017377011 || 0.9405798568062663 0.03815805711034295 ||

In [20]:
def calcualte_n_avg(data_box):
    return _np.average(data_box[0])
def calcualte_n_unc(data_box):
    return _np.std(data_box[0])/_np.sqrt(len(data_box[2])/block_size)

ns_avg = [calcualte_n_avg(data_box) for data_box in data_boxes]
ns_unc = [calcualte_n_unc(data_box) for data_box in data_boxes]

In [22]:
betas_final = []
Ns_final = []
ns_16 = []
ns_unc_16 = []
ns_20 = []
ns_unc_20 = []
ns_24 = []
ns_unc_24 = []
N=20

def avg(avgs_array, uncs_array, indexes):
    avgs = _np.array(avgs_array)[indexes]
    uncs = _np.array(uncs_array)[indexes]
    avgs = avgs[_np.nonzero(uncs)]
    uncs = uncs[_np.nonzero(uncs)]
    weights = uncs**-2
    weighted_average = (_np.sum(weights*avgs)/_np.sum(weights))
    final_unc = (1/_np.sqrt(_np.sum(weights)))
    return weighted_average, final_unc


for beta in set(betas):
    betas_final.append(beta)
    Ns_final.append(N)
    indexes_16 = _np.nonzero(_np.logical_and(_np.array(betas)==beta, _np.array(Ns)==16))
    indexes_20 = _np.nonzero(_np.logical_and(_np.array(betas)==beta, _np.array(Ns)==20))
    indexes_24 = _np.nonzero(_np.logical_and(_np.array(betas)==beta, _np.array(Ns)==24))
    print(indexes_16, indexes_20, indexes_24)
    avg_16, unc_16 = avg(ns_avg, ns_unc, indexes_16)
    avg_20, unc_20 = avg(ns_avg, ns_unc, indexes_20)
    avg_24, unc_24 = avg(ns_avg, ns_unc, indexes_24)
    ns_16.append(avg_16)
    ns_unc_16.append(unc_16)
    ns_20.append(avg_20)
    ns_unc_20.append(unc_20)
    ns_24.append(avg_24)
    ns_unc_24.append(unc_24)

sort_indexes = _np.argsort(betas_final)
betas_final = _np.array(betas_final)[sort_indexes]
ns_16 = _np.array(ns_16)[sort_indexes]
ns_unc_16 = _np.array(ns_unc_16)[sort_indexes]
ns_20 = _np.array(ns_20)[sort_indexes]
ns_unc_20 = _np.array(ns_unc_20)[sort_indexes]
ns_24 = _np.array(ns_24)[sort_indexes]
ns_unc_24 = _np.array(ns_unc_24)[sort_indexes]

(array([0, 1], dtype=int64),) (array([16, 17], dtype=int64),) (array([32, 33], dtype=int64),)
(array([2, 3], dtype=int64),) (array([18, 19], dtype=int64),) (array([34, 35], dtype=int64),)
(array([6, 7], dtype=int64),) (array([22, 23], dtype=int64),) (array([38, 39], dtype=int64),)
(array([4, 5], dtype=int64),) (array([20, 21], dtype=int64),) (array([36, 37], dtype=int64),)
(array([8, 9], dtype=int64),) (array([24, 25], dtype=int64),) (array([40, 41], dtype=int64),)
(array([10, 11], dtype=int64),) (array([26, 27], dtype=int64),) (array([42, 43], dtype=int64),)
(array([12, 13], dtype=int64),) (array([28, 29], dtype=int64),) (array([44, 45], dtype=int64),)
(array([14, 15], dtype=int64),) (array([30, 31], dtype=int64),) (array([46, 47], dtype=int64),)
[0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95]
[0.22079667 0.2706387  0.34868733 0.43341478 0.43175462 0.5229698
 0.61620204 0.79800292]
[0.01430051 0.01621802 0.01848305 0.02178716 0.0221155  0.02457274
 0.0269932  0.03327961]
[0.1567752  0.22106

In [25]:
for jj in range(len(chis_16)):
    print(betas_final[jj], ns_16[jj]/16**3, ns_unc_16[jj]/16**3, "||", ns_20[jj]/20**3, ns_unc_20[jj]/20**3, "||", ns_24[jj]/24**3, ns_unc_24[jj]/24**3)

0.88 0.08079625336793006 0.0001309294674845918 || 0.08006313875664903 9.355418712785525e-05 || 0.07966947335974801 7.326922212683818e-05
0.89 0.08103661610476764 0.00013359414110516844 || 0.0803412926581575 9.53458675757073e-05 || 0.07978024792193025 7.295937542326906e-05
0.9 0.0814691687213226 0.00013639599776566884 || 0.08068837378740643 9.879455440775063e-05 || 0.08013709759555505 7.717351663446838e-05
0.91 0.08198240933767424 0.00014054469083716695 || 0.08105612940992253 0.00010103195769780015 || 0.08061874124845324 7.918003136467407e-05
0.92 0.08188320592964683 0.00013926727030599085 || 0.08116349603010976 0.0001022721461059708 || 0.0806978004954859 8.094496599703398e-05
0.93 0.08244991978605147 0.00014008518049345452 || 0.08172291740975257 0.00010385110077803203 || 0.08133736823385564 8.021965446478423e-05
0.94 0.08305421934548339 0.00014284540117439362 || 0.08244349793001136 0.00010618586855010625 || 0.08217670399902853 8.122376791071424e-05
0.95 0.08438361381024463 0.0001419867

In [29]:
avgs = _np.array([ns_16[5]/(16**3), ns_20[5]/(20**3), ns_24[5]/(24**3)])
uncs = _np.array([ns_unc_16[5]/(16**3), ns_unc_20[5]/(20**3), ns_unc_24[5]/(24**3)])
avgs = avgs[_np.nonzero(uncs)]
uncs = uncs[_np.nonzero(uncs)]
weights = uncs**-2
weighted_average = (_np.sum(weights*avgs)/_np.sum(weights))
final_unc = (1/_np.sqrt(_np.sum(weights)))
print(weighted_average, final_unc)

0.08164646248476293 5.782425998905889e-05


In [19]:
for data_box in data_boxes:
    print(len(data_box[0]))

100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
100000
50000
